In [1]:
# Python script for analyses of "Antibody affinity birth through somatic hypermutation" publication.
# This pipeline is divided into X sections. At the beginning of each section there is a comment which indicates which figures of the publication are generated based on that section.

# input sequences for these analyses are uploaded in data folder. By a successful run, the result of each section will be saved in output folder.
print('Running...')
import re
import operator

import os
#import sys
import pandas as pd
import numpy as np

import time
import itertools
import matplotlib.pyplot as plt
import glob
#import logomaker #https://logomaker.readthedocs.io

# Functions
def display_big():

    # df = pd.DataFrame()
    # pd.options.display.max_colwidth = 2000
    pd.set_option('display.max_rows', 10)
    pd.set_option('display.max_columns', 200)
    pd.set_option('display.width', 1000)

display_big()

Running...


In [2]:
data_folder='../data'

input_folder = os.getenv('VAR_IN_FOLDER', f"{data_folder}/input")
output_folder = os.getenv('VAR_OUT_FOLDER', f"{data_folder}/output")

In [3]:
def set_output_folder(section_output):
    output_folder=data_folder+'/output/'+section_output

    if not os.path.isdir(output_folder): # make output folder if it doesn't exist
        os.makedirs(output_folder)
    return(output_folder)

In [4]:
# Section1: preparation

output_folder_prep=set_output_folder('1_prep')
output_folder_num_miss=set_output_folder('2_num_miss')
output_folder_freq_pos=set_output_folder('3_freq_per_position')
output_folder_donuts=set_output_folder('4_donuts')
output_folder_seq_logos=set_output_folder('5_seq_logos')
output_folder_rs_prep=set_output_folder('6_prep_rs')
output_folder_rs=set_output_folder('7_rs')
output_folder_prep_plots=set_output_folder('8_prep_plots')
output_folder_scatter_plots=set_output_folder('9_scatter_plots')


In [5]:
del_sign='-'
ambiguity_sign='.'
aas_dic={'AAA':'K','AAC':'N','AAT':'N','AAG':'K','ACA':'T','ACC':'T','ACT':'T','ACG':'T','ATA':'I','ATC':'I',\
        'ATT':'I','ATG':'M','AGA':'R','AGC':'S','AGT':'S','AGG':'R','CAA':'Q','CAC':'H','CAT':'H','CAG':'Q',\
        'CCA':'P','CCC':'P','CCT':'P','CCG':'P','CTA':'L','CTC':'L','CTT':'L','CTG':'L','CGA':'R','CGC':'R',\
        'CGT':'R','CGG':'R','TAA':'*','TAC':'Y','TAT':'Y','TAG':'*','TCA':'S','TCC':'S','TCT':'S','TCG':'S',\
        'TTA':'L','TTC':'F','TTT':'F','TTG':'L','TGA':'*','TGC':'C','TGT':'C','TGG':'W','GAA':'E','GAC':'D',\
        'GAT':'D','GAG':'E','GCA':'A','GCC':'A','GCT':'A','GCG':'A','GTA':'V','GTC':'V','GTT':'V','GTG':'V',\
        'GGA':'G','GGC':'G','GGT':'G','GGG':'G','---':del_sign}
aas_list=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', '*', del_sign]
aas_chemistry_list=['I', 'V', 'L', 'F', 'C', 'M', 'A', 'W', 'G', 'T', 'S', 'Y', 'P', 'H', 'N', 'D', 'Q', 'E', 'K', 'R']
nts_list=['A', 'C', 'G', 'T', del_sign, ambiguity_sign]

In [6]:
modes_dic={ 1 : {'mouse' : 'B18-383', 'datasets': ['OVA', 'APC', 'CGG', 'Passenger'], 'chain':'VH'},
            2 : {'mouse' : 'HA-WT', 'datasets': ['OVA', 'APC', 'CGG', 'mix'], 'chain':'VH'},
           
            3 : {'mouse' : 'B18-383', 'datasets': ['OVA-CTLA4', 'OVA-Isotype'], 'chain':'VH'},
            4 : {'mouse' : 'HA-uMT', 'datasets': ['OVA-CTLA4', 'OVA-Isotype'], 'chain':'VH'},
            5 : {'mouse' : 'HA-WT', 'datasets': ['CGG-CTLA4', 'CGG-Isotype'], 'chain':'VH'},

            6 : {'mouse' : 'B18-383', 'datasets': ['OVA-CTLA4', 'OVA-Isotype'], 'chain':'VL'},
            7 : {'mouse' : 'HA-uMT', 'datasets': ['OVA-CTLA4', 'OVA-Isotype'], 'chain':'VL'},
            8 : {'mouse' : 'HA-WT', 'datasets': ['CGG-CTLA4', 'CGG-Isotype'], 'chain':'VL'}}

In [7]:
# Section7: frquency and privacy index calculations and scatter plots
# used in Figs 4D, 4E, 4G, 5I, 6H, S4D, S5C

for mode in modes_dic.keys():

    df_nts=pd.read_csv('{}/dfs_expanded_aas_excluded.tsv'.format(output_folder_prep), sep='\t', header=0, low_memory=False)
    mouse=modes_dic[mode]['mouse']
    datasets=modes_dic[mode]['datasets']
    chain=modes_dic[mode]['chain']
    datasets


    df_nts

    df_aa=pd.read_csv('{}/df_aa_mode{}_{}.tsv'.format(output_folder_prep_plots, mode, chain), sep='\t', header=0, low_memory=False, index_col=0)
    df_aa=df_aa.iloc[:-3,:].copy()
    
    if mode == 1:
        df_nts=df_nts.loc[
        (df_nts['label']=='LateGC_B18-383_OVA_VH_-') |\
        (df_nts['label']=='LateGC_B18-383_APC_VH_-') |\
        (df_nts['label']=='LateGC_B18-383_CGG_VH_-') |\
        (df_nts['label']=='Published_B18_Passenger_VH_-') \
        ,].copy()
        print(set(df_nts['label']))
        chain='VH'
    
    elif mode == 2:
        df_nts=df_nts.loc[
        (df_nts['label']=='LateGC_HA-uMT_OVA_VH_0-1') |\
        (df_nts['label']=='LateGC_HA-uMT_APC_VH_0-1') |\
        (df_nts['label']=='LateGC_HA-uMT_CGG_VH_0-1') |\
        (df_nts['label']=='LateGC_HA-WT_mix_VH_1-1') \
        ,].copy()
        print(set(df_nts['label']))
        chain='VH'
            
    elif mode == 3:
        df_nts=df_nts.loc[
        (df_nts['label']=='LateGC_B18-383_OVA-CTLA4_VH_-') |\
        (df_nts['label']=='LateGC_B18-383_OVA-Isotype_VH_-') \
        ,].copy()
        print(set(df_nts['label']))
        chain='VH'
    
    elif mode == 4:
        df_nts=df_nts.loc[
        (df_nts['label']=='LateGC_HA-uMT_OVA-CTLA4_VH_0-1') |\
        (df_nts['label']=='LateGC_HA-uMT_OVA-Isotype_VH_0-1') \
        ,].copy()
        print(set(df_nts['label']))
        chain='VH'
    
    elif mode == 5:
        df_nts=df_nts.loc[
        (df_nts['label']=='LateGC_HA-WT_CGG-CTLA4_VH_1-1000') |\
        (df_nts['label']=='LateGC_HA-WT_CGG-Isotype_VH_1-1000') \
        ,].copy()
        print(set(df_nts['label']))    	
        chain='VH'
    
    elif mode == 6:
        df_nts=df_nts.loc[
        (df_nts['label']=='LateGC_B18-383_OVA-CTLA4_VL_-') |\
        (df_nts['label']=='LateGC_B18-383_OVA-Isotype_VL_-') \
        ,].copy()
        print(set(df_nts['label']))
        chain='VL'
    
    elif mode == 7:
        df_nts=df_nts.loc[
        (df_nts['label']=='LateGC_HA-uMT_OVA-CTLA4_VL_0-1') |\
        (df_nts['label']=='LateGC_HA-uMT_OVA-Isotype_VL_0-1') \
        ,].copy()
        print(set(df_nts['label']))
        chain='VL'
    
    elif mode == 8:
        df_nts=df_nts.loc[
        (df_nts['label']=='LateGC_HA-WT_CGG-CTLA4_VL_1-1000') |\
        (df_nts['label']=='LateGC_HA-WT_CGG-Isotype_VL_1-1000') \
        ,].copy()
        print(set(df_nts['label']))    	
        chain='VL'

    print('Mode :', mode, mouse, chain, datasets)
    
    #Mode 1
    # 5136 AA MM + AA nMM
    # 4810 AA MM

    print(set(df_nts['mouse']), set(df_nts['chain']), set(df_nts['dataset']))

    for dataset in datasets:
        print('{}:{}'.format(dataset, len(df_nts[df_nts['dataset']==dataset])))

    # 2:3
    # 3:7
    # 4:15

    all_combinations=list(np.arange(1.0,16.0)) #15

    if len(datasets)==2:
        cases_list_wanted_all={'shareds':[4.0], 'uniques':[1.0, 2.0], 'oxaxcx':[], 'remainders':[]}

    if len(datasets)==3:
        cases_list_wanted_all={'shareds':[7.0], 'uniques':[1.0, 2.0, 3.0], 'oxaxcx':[5.0, 6.0], 'remainders':[4.0]}

    if len(datasets)==4:
        cases_list_wanted_all={'shareds':[15.0], 'uniques':[1.0, 2.0, 3.0, 8.0], 'oxaxcx':[9.0, 10.0, 11.0], 'remainders':[4.0, 5.0, 6.0, 7.0, 12.0, 13.0, 14.0]}

    dfs_cases_dic={}
    dfs_cases_num_dic={}
    for case, list_wanted in cases_list_wanted_all.items():

        dic_num_blocks={}

        list_exclusion=all_combinations.copy()
        df_aa_now=df_aa.copy()

        for i in list_wanted:
            list_exclusion.remove(i)

        for j in list_exclusion:
            df_aa_now.replace({ j : np.nan}, inplace=True)

        df_aa_now=df_aa_now.iloc[:-2,:]

        for n in list_wanted:
            dic_num_blocks[n]=(df_aa_now == n).sum().sum()
        # print(case, dic_num_blocks)
        dfs_cases_dic[case]=df_aa_now
        dfs_cases_num_dic[case]=dic_num_blocks
        # display(df_aa_now)

    datasets

    dfs_cases_num_dic

    pd.DataFrame.from_dict(dfs_cases_num_dic, orient='index').T.to_csv('{}/mode{}_numbers_{}.tsv'.format(output_folder_scatter_plots, mode, chain), sep = '\t', index=True)

    dfs_cases_dic.keys()

    dfs_cases_dic['shareds']

    df_pos_shareds = dfs_cases_dic['shareds'].T
    df_pos_uniques = dfs_cases_dic['uniques'].T
    df_pos_oxaxcx = dfs_cases_dic['oxaxcx'].T
    df_pos_remainders = dfs_cases_dic['remainders'].T

    # df_pos_uniques = pd.read_csv('../data/avneesh/affinity_birth/output/F005/mode{}-1_2_3_8.tsv'.format(mode), sep='\t', header=0, low_memory=False, index_col=0)
    # df_pos_all = pd.read_csv('../data/avneesh/affinity_birth/output/F005/mode{}-1_2_3_4_5_6_7_8_9_10_11_12_13_14_15.tsv'.format(mode), sep='\t', header=0, low_memory=False, index_col=0)

    # df_nts = pd.read_csv('../data/avneesh/affinity_birth/output/F002/all_nt_seq_based.tsv', sep='\t', header=0, low_memory=False) #, index_col='codon'
    # df_nts=df_nts[df_nts['frameshift']==False]
    # df_nts=df_nts[df_nts['stopcodon']==False]

    set(df_nts['dataset'])

    # if mode==5:
    #     df_now=pd.concat([df_nts[df_nts['dataset']=='1_1_OVA'], pd.concat([df_nts[df_nts['dataset']=='1_1_APC'],df_nts[df_nts['dataset']=='1_1_CGG']])])
    #     df_now['dataset']='HA-WT'
    #     df_nts=pd.concat([df_nts,df_now])



    mouse

    set(df_nts['mouse'])

    aa_ref_seq=df_nts['ref_aa'].values[-1]

    len_ref=len(df_nts['ref_aa'].values[-1])

    df_nts

    multiplier_size=22 # To increase the size of shapes on the plot
    # df_nts.to_csv('{}/mode{}_relevant_dfs.tsv'.format(output_folder, mode), sep = '\t', index=True)

    # To translate AAs names to numbers to sort them by chemical properties
    aa_D = dict()
    aa_L=aas_chemistry_list
    for y, aa in enumerate(aa_L):
        aa_D[aa]=20-y
    print(aa_D)

    list_frequency=['{}_frequency'.format(ds) for ds in datasets]
    list_plot_size=['{}_plot_size'.format(ds) for ds in datasets]
    list_selec_uneven=['{}_selec_uneven'.format(ds) for ds in datasets]

    print(list_frequency)
    print(list_plot_size)
    print(list_selec_uneven)

    df_pos_uniques

    df_aa=df_aa.T

    df_nts['unique_num']=0
    df_nts['shared_num']=0

    # Uniques dataframe
    myDict = dict()

    ds_unique_names={1.0: 0, 2.0: 1, 3.0:2, 8.0:3} # For converting indices from 1238 to 0123

    for p, aa in itertools.product(*[df_aa.index,df_aa.columns[:-2]]):
        v = df_aa.loc[p,aa]
        # if v not in [1.0,2.0,3.0,8.0]: continue
        if v not in cases_list_wanted_all['uniques']: continue

        # myDict['{}-{}'.format(p,aa)] = {'p': p, 'aa' : aa}
        myDict['{}-{}'.format(p,aa)] = {'p': p, 'aa': aa, 'aa_n' : aa_D[aa]} # Translate AAs names to numbers to sort them by chemical properties

        n = ds_unique_names[v] # Convert indexing 1238 to 0123

        for i in df_nts[(df_nts['A{}'.format(p)]==aa) & (df_nts['dataset']==datasets[n])]['header'].index:
            # print(i)
            df_nts.loc[i,'unique_num']+=1

        instances = len(df_nts[(df_nts['A{}'.format(p)]==aa) & (df_nts['dataset']==datasets[n])])
        df_size = len(df_nts[df_nts["dataset"]==datasets[n]])
        df_frequency = instances/df_size*100

        myDict['{}-{}'.format(p,aa)]['{}_instances'.format(datasets[n])] = instances
        myDict['{}-{}'.format(p,aa)]['{}_size'.format(datasets[n])] = df_size

        myDict['{}-{}'.format(p,aa)]['{}_frequency'.format(datasets[n])] = df_frequency
        # myDict['{}-{}'.format(p,aa)]['{}_plot_size'.format(datasets[n])] = df_frequency*multiplier_size
        myDict['{}-{}'.format(p,aa)]['{}_selec_uneven'.format(datasets[n])] = 0.1

    df_all_uniques=pd.DataFrame.from_dict(myDict).transpose()
    # display(df_all_uniques)
    df_all_uniques['status']='uniques'

    df_all_uniques['p']=df_all_uniques['p'].astype('int')+1 # Change the zero-indexing to one-indexing
    df_all_uniques.to_csv('{}/mode{}_uniques_{}.tsv'.format(output_folder_scatter_plots, mode, chain), sep = '\t', index=True)

    df_all_uniques

    myDict = dict()

    for p, aa in itertools.product(*[df_aa.index, df_aa.columns[:-2]]):
        total_frequency = 0
        v = df_aa.loc[p,aa]
        # if v not in [9.0,10.0,11.0]: continue
        if v not in cases_list_wanted_all['oxaxcx']: continue

        # myDict['{}-{}'.format(p,aa)] = {'p': p, 'aa' : aa}
        myDict['{}-{}'.format(p,aa)] = {'p': p, 'aa' : aa, 'aa_n' : aa_D[aa]}
        for n in range(0, len(datasets)):
            # print(n)
            instances = len(df_nts[(df_nts['A{}'.format(p)]==aa) & (df_nts['dataset']==datasets[n])])
            df_size = len(df_nts[df_nts["dataset"]==datasets[n]])
            df_frequency = instances/df_size*100
            total_frequency+=df_frequency

            myDict['{}-{}'.format(p,aa)]['{}_instances'.format(datasets[n])] = instances
            myDict['{}-{}'.format(p,aa)]['{}_size'.format(datasets[n])] = df_size

            myDict['{}-{}'.format(p,aa)]['{}_frequency'.format(datasets[n])] = df_frequency
            # myDict['{}-{}'.format(p,aa)]['{}_plot_size'.format(datasets[n])] = df_frequency*multiplier_size

        for n in range(0, len(datasets)):
            myDict['{}-{}'.format(p,aa)]['{}_selec_uneven'.format(datasets[n])] = myDict['{}-{}'.format(p,aa)]['{}_frequency'.format(datasets[n])]/total_frequency

    df_oxaxcx=pd.DataFrame()

    if len(myDict)!=0:
        df_oxaxcx=pd.DataFrame.from_dict(myDict).transpose()

        df_oxaxcx['p']=df_oxaxcx['p'].astype('int')+1 # Change the zero-indexing to one-indexing

        df_oxaxcx['status']='oxaxcx'
        df_oxaxcx.to_csv('{}/mode{}_oxaxcx_{}.tsv'.format(output_folder_scatter_plots, mode, chain), sep = '\t', index=True)

    df_oxaxcx

    # Remainder Shareds dataframe
    myDict = dict()

    for p, aa in itertools.product(*[df_aa.index, df_aa.columns[:-2]]):
        total_frequency = 0
        v = df_aa.loc[p,aa]
        if v not in cases_list_wanted_all['remainders']: continue

        # myDict['{}-{}'.format(p,aa)] = {'p': p, 'aa' : aa}
        myDict['{}-{}'.format(p,aa)] = {'p': p, 'aa' : aa, 'aa_n' : aa_D[aa]}
        for n in range(0, len(datasets)):
            # print(n)
            instances = len(df_nts[(df_nts['A{}'.format(p)]==aa) & (df_nts['dataset']==datasets[n])])
            df_size = len(df_nts[df_nts["dataset"]==datasets[n]])
            df_frequency = instances/df_size*100
            total_frequency+=df_frequency

            myDict['{}-{}'.format(p,aa)]['{}_instances'.format(datasets[n])] = instances
            myDict['{}-{}'.format(p,aa)]['{}_size'.format(datasets[n])] = df_size

            myDict['{}-{}'.format(p,aa)]['{}_frequency'.format(datasets[n])] = df_frequency
            # myDict['{}-{}'.format(p,aa)]['{}_plot_size'.format(datasets[n])] = df_frequency*multiplier_size

        for n in range(0, len(datasets)):
            myDict['{}-{}'.format(p,aa)]['{}_selec_uneven'.format(datasets[n])] = myDict['{}-{}'.format(p,aa)]['{}_frequency'.format(datasets[n])]/total_frequency

    df_remainders=pd.DataFrame()

    if len(myDict)!=0:

        df_remainders=pd.DataFrame.from_dict(myDict).transpose()

        df_remainders['p']=df_remainders['p'].astype('int')+1 # Change the zero-indexing to one-indexing

        df_remainders['status']='remainders'
        df_remainders.to_csv('{}/mode{}_remainders_{}.tsv'.format(output_folder_scatter_plots, mode, chain), sep = '\t', index=True)

    df_remainders

    # Shareds dataframe
    myDict = dict()

    for p, aa in itertools.product(*[df_aa.index, df_aa.columns[:-2]]):
        total_frequency = 0
        v = df_aa.loc[p,aa]
        if v not in cases_list_wanted_all['shareds']: continue

        # myDict['{}-{}'.format(p,aa)] = {'p': p, 'aa' : aa}
        myDict['{}-{}'.format(p,aa)] = {'p': p, 'aa' : aa, 'aa_n' : aa_D[aa]}
        for n in range(0, len(datasets)):
            # print(n)

            for i in df_nts[(df_nts['A{}'.format(p)]==aa) & (df_nts['dataset']==datasets[n])]['header'].index:
                df_nts.loc[i,'shared_num']+=1
                # print(datasets[n],i)
            instances = len(df_nts[(df_nts['A{}'.format(p)]==aa) & (df_nts['dataset']==datasets[n])])

            df_size = len(df_nts[df_nts["dataset"]==datasets[n]])
            df_frequency = instances/df_size*100
            total_frequency+=df_frequency

            myDict['{}-{}'.format(p,aa)]['{}_instances'.format(datasets[n])] = instances
            myDict['{}-{}'.format(p,aa)]['{}_size'.format(datasets[n])] = df_size

            myDict['{}-{}'.format(p,aa)]['{}_frequency'.format(datasets[n])] = df_frequency
            # myDict['{}-{}'.format(p,aa)]['{}_plot_size'.format(datasets[n])] = df_frequency*multiplier_size

        for n in range(0, len(datasets)):
            myDict['{}-{}'.format(p,aa)]['{}_selec_uneven'.format(datasets[n])] = myDict['{}-{}'.format(p,aa)]['{}_frequency'.format(datasets[n])]/total_frequency
    df_all_shareds=pd.DataFrame.from_dict(myDict).transpose()
    df_all_shareds['status']='shareds'

    df_all_shareds['p']=df_all_shareds['p'].astype('int')+1 # Change the zero-indexing to one-indexing
    df_all_shareds.to_csv('{}/mode{}_shareds_{}.tsv'.format(output_folder_scatter_plots, mode, chain), sep = '\t', index=True)
    df_all_shareds

    grouping=df_nts.groupby(by='dataset')
    dfs_unique_num=pd.DataFrame()
    dfs_shared_num=pd.DataFrame()

    for ds, df_now in grouping:
        cols=['{}_unique_num'.format(ds), '{}_shared_num'.format(ds)]
        df_now.rename(columns={'unique_num':cols[0], 'shared_num':cols[1]}, inplace=True)
        df_now=df_now[[cols[0], cols[1]]]

        df_now=df_now.sort_values(by=cols[0], ascending=False)
        df_now.reset_index(inplace=True, drop=True)
        dfs_unique_num=pd.concat([dfs_unique_num, df_now[cols[0]]], axis=1)

        df_now=df_now.sort_values(by=cols[1], ascending=False)
        df_now.reset_index(inplace=True, drop=True)
        dfs_shared_num=pd.concat([dfs_shared_num, df_now[cols[1]]], axis=1)

    list_dfs_unique_num=['{}_unique_num'.format(ds) for ds in datasets]
    dfs_unique_num[list_dfs_unique_num].to_csv('{}/mode{}_unique_num_{}_aa.tsv'.format(output_folder_scatter_plots, mode, chain), sep = '\t', index=False)

    list_dfs_shared_num=['{}_shared_num'.format(ds) for ds in datasets]
    dfs_shared_num[list_dfs_shared_num].to_csv('{}/mode{}_shared_num_{}_aa.tsv'.format(output_folder_scatter_plots, mode, chain), sep = '\t', index=False)

    # raise Exception("Stop!")

    n=10
    for c in list_frequency:
        print('top {} of:{}'.format(n, c))
        df_now=df_all_shareds.dropna().sort_values(by=c, ascending=False).iloc[0:n,]
        df_now=df_now[['p', 'aa']+list_frequency]
        # display(df_now)

        df_now.to_csv('{}/mode{}_top_{}_shared_{}_{}.tsv'.format(output_folder_scatter_plots, mode, n, c, chain), sep = '\t', index=False)


    df_all_shareds

    # Calculating min/max ranges for uniform size/intensity plotting
    min_frequency_uq=df_all_uniques[list_frequency].min(axis=0).min()
    max_frequency_uq=df_all_uniques[list_frequency].max(axis=0).max()

    min_selec_uneven_uq=df_all_uniques[list_selec_uneven].min(axis=0).min()
    max_selec_uneven_uq=df_all_uniques[list_selec_uneven].max(axis=0).max()

    # print('uniques :', min_frequency_uq, max_frequency_uq, min_selec_uneven_uq, max_selec_uneven_uq)

    min_frequency_shd=df_all_shareds[list_frequency].min(axis=0).min()
    max_frequency_shd=df_all_shareds[list_frequency].max(axis=0).max()

    min_selec_uneven_shd=df_all_shareds[list_selec_uneven].min(axis=0).min()
    max_selec_uneven_shd=df_all_shareds[list_selec_uneven].max(axis=0).max()

    # print('shareds :', min_frequency_shd, max_frequency_shd, min_selec_uneven_shd, max_selec_uneven_shd)

    min_frequency=min(min_frequency_uq, min_frequency_shd)
    max_frequency=max(max_frequency_uq, max_frequency_shd)

    min_selec_uneven=min(min_selec_uneven_uq, min_selec_uneven_shd)
    max_selec_uneven=max(max_selec_uneven_uq, max_selec_uneven_shd)

    print('# Mode :', mode, mouse, chain, datasets,\
          '\n# uniques :', min_frequency_uq, max_frequency_uq, min_selec_uneven_uq, max_selec_uneven_uq,\
          '\n# shareds :', min_frequency_shd, max_frequency_shd, min_selec_uneven_shd, max_selec_uneven_shd,\
          '\n# uq/shd :', min_frequency, max_frequency, min_selec_uneven, max_selec_uneven)

    multi_index = pd.MultiIndex.from_tuples([('min', 'frequency'),('max', 'frequency'),
                                             ('min', 'selec_uneven'),('max', 'selec_uneven')])

    df_min_max = pd.DataFrame(index=multi_index, columns=['uniques','shareds', 'oxaxcx', 'remainders'])

    df_min_max.loc[('min', 'frequency'), 'uniques'] = min_frequency_uq
    df_min_max.loc[('max', 'frequency'), 'uniques'] = max_frequency_uq
    df_min_max.loc[('min', 'selec_uneven'), 'uniques'] = min_selec_uneven_uq
    df_min_max.loc[('max', 'selec_uneven'), 'uniques'] = max_selec_uneven_uq

    df_min_max.loc[('min', 'frequency'), 'shareds'] = min_frequency_shd
    df_min_max.loc[('max', 'frequency'), 'shareds'] = max_frequency_shd
    df_min_max.loc[('min', 'selec_uneven'), 'shareds'] = min_selec_uneven_shd
    df_min_max.loc[('max', 'selec_uneven'), 'shareds'] = max_selec_uneven_shd

    df_min_max.loc[('min', 'frequency'), 'oxaxcx'] = min_frequency_shd
    df_min_max.loc[('max', 'frequency'), 'oxaxcx'] = max_frequency_shd
    df_min_max.loc[('min', 'selec_uneven'), 'oxaxcx'] = min_selec_uneven_shd
    df_min_max.loc[('max', 'selec_uneven'), 'oxaxcx'] = max_selec_uneven_shd

    df_min_max.loc[('min', 'frequency'), 'remainders'] = min_frequency_shd
    df_min_max.loc[('max', 'frequency'), 'remainders'] = max_frequency_shd
    df_min_max.loc[('min', 'selec_uneven'), 'remainders'] = min_selec_uneven_shd
    df_min_max.loc[('max', 'selec_uneven'), 'remainders'] = max_selec_uneven_shd

    df_min_max.to_csv('{}/mode{}_df_min_max_{}.tsv'.format(output_folder_scatter_plots, mode, chain), sep = '\t', index=True)
    df_min_max

    min_frequency=0
    max_frequency=100
    min_selec_uneven=0
    max_selec_uneven=1.25

    intensity_range=np.linspace(min_selec_uneven, max_selec_uneven, 6, endpoint=True)
    intensity_range=[1.5,1,.75,.5,.25,.01]
    size_range=np.linspace(min_frequency, max_frequency, 6, endpoint=True)
    size_range=[150,100,75,50,25,1]
    aa_range=[3,8,10,12,14,16]

    intensity_range

    size_range

    dfs=[df_all_shareds, df_oxaxcx, df_remainders, df_all_uniques]
    statuses=['shareds', 'oxaxcx', 'remainders', 'uniques']

    for p, df_now, status in zip([len_ref+3, len_ref+5, len_ref+7, len_ref+9], dfs, statuses):

        for P, A, S, I in zip([p]*6, aa_range, size_range, [1]*6):
            if status=='uniques': I=0.1
            df_now.loc['m{}S_{}I-{}'.format(S,I,P), 'p'] = P
            df_now.loc['m{}S_{}I-{}'.format(S,I,P), 'aa_n'] = A
            df_now.loc['m{}S_{}I-{}'.format(S,I,P), 'status'] = status

            for c in list_frequency:
                df_now.loc['m{}S_{}I-{}'.format(S,I,P), c] = S
            for c in list_selec_uneven:
                df_now.loc['m{}S_{}I-{}'.format(S,I,P), c] = I
        # display(df_now)
        for P, A, S, I in zip([p+9]*6, aa_range, [max_frequency*2/3]*6, intensity_range):
            if status=='uniques':
                I=0.1
                A=8
            df_now.loc['m{}S_{}I-{}'.format(S,I,P), 'p'] = P
            df_now.loc['m{}S_{}I-{}'.format(S,I,P), 'aa_n'] = A
            df_now.loc['m{}S_{}I-{}'.format(S,I,P), 'status'] = status

            for c in list_frequency:
                df_now.loc['m{}S_{}I-{}'.format(S,I,P), c] = S
            for c in list_selec_uneven:
                df_now.loc['m{}S_{}I-{}'.format(S,I,P), c] = I

    # Calculate shape sizes for ploting
    for n, ds_pl in itertools.product(*[range(0, len(datasets)), dfs]):
        ds_pl['{}_plot_size'.format(datasets[n])]=ds_pl['{}_frequency'.format(datasets[n])]*multiplier_size

    # df_all_shareds[df_all_shareds['p']==len_ref+3]

    # df_oxaxcx[df_oxaxcx['p']==len_ref+5]

    # df_remainders[df_remainders['p']==len_ref+7]

    # df_all_uniques[df_all_uniques['p']==len_ref+9]

    plot_dic={'shareds': df_all_shareds, 'uniques': df_all_uniques, 'oxaxcx': df_oxaxcx, 'remainders': df_remainders}

    # 'Orange' is not a valid value for cmap; supported values are 'Accent', 'Accent_r', 'Blues', 'Blues_r', 'BrBG', 'BrBG_r', 'BuGn', 'BuGn_r', 'BuPu', 'BuPu_r',
    # 'CMRmap', 'CMRmap_r', 'Dark2', 'Dark2_r', 'GnBu', 'GnBu_r', 'Greens', 'Greens_r', 'Greys', 'Greys_r', 'OrRd', 'OrRd_r', 'Oranges', 'Oranges_r', 'PRGn', 'PRGn_r',
    # 'Paired', 'Paired_r', 'Pastel1', 'Pastel1_r', 'Pastel2', 'Pastel2_r', 'PiYG', 'PiYG_r', 'PuBu', 'PuBuGn', 'PuBuGn_r', 'PuBu_r', 'PuOr', 'PuOr_r', 'PuRd', 'PuRd_r',
    # 'Purples', 'Purples_r', 'RdBu', 'RdBu_r', 'RdGy', 'RdGy_r', 'RdPu', 'RdPu_r', 'RdYlBu', 'RdYlBu_r', 'RdYlGn', 'RdYlGn_r', 'Reds', 'Reds_r', 'Set1', 'Set1_r', 'Set2',
    # 'Set2_r', 'Set3', 'Set3_r', 'Spectral', 'Spectral_r', 'Wistia', 'Wistia_r', 'YlGn', 'YlGnBu', 'YlGnBu_r', 'YlGn_r', 'YlOrBr', 'YlOrBr_r', 'YlOrRd', 'YlOrRd_r', 'afmhot',
    # 'afmhot_r', 'autumn', 'autumn_r', 'binary', 'binary_r', 'bone', 'bone_r', 'brg', 'brg_r', 'bwr', 'bwr_r', 'cividis', 'cividis_r', 'cool', 'cool_r', 'coolwarm', 'coolwarm_r',
    # 'copper', 'copper_r', 'crest', 'crest_r', 'cubehelix', 'cubehelix_r', 'flag', 'flag_r', 'flare', 'flare_r', 'gist_earth', 'gist_earth_r', 'gist_gray', 'gist_gray_r', 'gist_heat',
    # 'gist_heat_r', 'gist_ncar', 'gist_ncar_r', 'gist_rainbow', 'gist_rainbow_r', 'gist_stern', 'gist_stern_r', 'gist_yarg', 'gist_yarg_r', 'gnuplot', 'gnuplot2', 'gnuplot2_r', 'gnuplot_r',
    # 'gray', 'gray_r', 'hot', 'hot_r', 'hsv', 'hsv_r', 'icefire', 'icefire_r', 'inferno', 'inferno_r', 'jet', 'jet_r', 'magma', 'magma_r', 'mako', 'mako_r', 'nipy_spectral', 'nipy_spectral_r',
    # 'ocean', 'ocean_r', 'pink', 'pink_r', 'plasma', 'plasma_r', 'prism', 'prism_r', 'rainbow', 'rainbow_r', 'rocket', 'rocket_r', 'seismic', 'seismic_r', 'spring', 'spring_r', 'summer', 'summer_r',
    # 'tab10', 'tab10_r', 'tab20', 'tab20_r', 'tab20b', 'tab20b_r', 'tab20c', 'tab20c_r', 'terrain', 'terrain_r', 'turbo', 'turbo_r', 'twilight', 'twilight_r', 'twilight_shifted', 'twilight_shifted_r',
    # 'viridis', 'viridis_r', 'vlag', 'vlag_r', 'winter', 'winter_r'

    Legend=False
    for ds in datasets:
        print(ds)
        plt.rc('font', size=30)

        kwargs  =   {'edgecolor' : "gray", # for edge color
                     'linewidth' : 0.3, # line width of spot
                     'linestyle' : '-', # line style of spot
                    }

        cmap = {'shareds':'Reds', 'uniques':'spring', 'oxaxcx':'Greens', 'remainders':'Blues'}

        if ds==datasets[0] or ds==datasets[-1]: fig, ax = plt.subplots(figsize=(45,8.5))
        else: fig, ax = plt.subplots(figsize=(45,8))



        for status, df_plot in plot_dic.items(): #df shareds or uniques
            df_plot = df_plot[['p', 'aa_n', 'status', '{}_plot_size'.format(ds), '{}_selec_uneven'.format(ds)]].dropna() #, '{}_frequency'.format(ds)
            # df.to_csv('{}/mode{}_{}_df.tsv'.format(output_folder, mode, ds), sep = '\t', index=True)

            # display(df_plot)
            # print(len(df_plot))
            # print(status)
            # s=np.arange(1,111)

            # xx=df_plot['{}_plot_size'.format(ds)]
            # xx=[x * 30 for x in xx]
            # print(xx, len(xx))
            ax.scatter(
                df_plot['p'], df_plot['aa_n'],
                s=list(df_plot['{}_plot_size'.format(ds)]), c=df_plot['{}_selec_uneven'.format(ds)], cmap=cmap[status], **kwargs,# color=color,, marker=marker,
                label=status,
            )

        plt.xticks([])

        if ds==datasets[0]:

            plt.tick_params(which='both', top=True, labeltop=True, bottom=False, labelbottom=False)
            # plt.tick_params(which='minor', top=True, labeltop=True, bottom=False, labelbottom=False)
            plt.xticks(ticks=range(1,len_ref+1, 1), labels=list(aa_ref_seq), minor=False)
            plt.xticks(ticks=range(2,len_ref+1, 2), minor=True)
            plt.xticks(fontsize=40)
            plt.xticks(fontsize=40)

        if ds==datasets[-1]:

            plt.tick_params(which='both', top=False, labeltop=False, bottom=True, labelbottom=True)
            # plt.tick_params(which='minor', top=True, labeltop=True, bottom=False, labelbottom=False)
            plt.xticks(ticks=range(0,len_ref+0, 5), labels=range(0,len_ref+0, 5), minor=False)
            plt.xticks(ticks=range(1,len_ref+1, 1), minor=True)
            plt.xticks(fontsize=40)

        # plt.xticks([])
        # if ds=='Passenger':
        #     plt.xticks(range(0,len_ref+1,2))
        #     # plt.minorticks_on(labelbottom=False)
        # if ds=='OVA':
        #     plt.xticks(range(1,len_ref+1), aa_ref_seq)
        #     plt.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)

        plt.yticks(range(20,0,-1), aa_L)
        plt.xlim([0.5, len_ref+1])
        if Legend==True: plt.xlim([0.5, len_ref+20])

        plt.tight_layout()
        # plt.title("title")
        # plt.xlabel('Positions (1-{})'.format(len_ref), )
        # plt.xlabel('')
        plt.ylabel('Amino acids', weight='bold')
        plt.ylabel('')

        # plt.grid(color='lightgray', linestyle='-', linewidth=0.5, axis='both')
        plt.savefig('{}/mode{}-{}-{}.jpg'.format(output_folder_scatter_plots, mode, ds, chain, dpi=150))
        plt.close(fig)
        time.sleep(0.1)
        plt.pause(0.0001)


# raise Exception("Stop!")


print('Finished successfully!')    

{'Published_B18_Passenger_VH_-', 'LateGC_B18-383_APC_VH_-', 'LateGC_B18-383_CGG_VH_-', 'LateGC_B18-383_OVA_VH_-'}
Mode : 1 B18-383 VH ['OVA', 'APC', 'CGG', 'Passenger']
{'B18', 'B18-383'} {'VH'} {'Passenger', 'OVA', 'APC', 'CGG'}
OVA:141
APC:95
CGG:48
Passenger:6461
{'I': 20, 'V': 19, 'L': 18, 'F': 17, 'C': 16, 'M': 15, 'A': 14, 'W': 13, 'G': 12, 'T': 11, 'S': 10, 'Y': 9, 'P': 8, 'H': 7, 'N': 6, 'D': 5, 'Q': 4, 'E': 3, 'K': 2, 'R': 1}
['OVA_frequency', 'APC_frequency', 'CGG_frequency', 'Passenger_frequency']
['OVA_plot_size', 'APC_plot_size', 'CGG_plot_size', 'Passenger_plot_size']
['OVA_selec_uneven', 'APC_selec_uneven', 'CGG_selec_uneven', 'Passenger_selec_uneven']
top 10 of:OVA_frequency
top 10 of:APC_frequency
top 10 of:CGG_frequency
top 10 of:Passenger_frequency
# Mode : 1 B18-383 VH ['OVA', 'APC', 'CGG', 'Passenger'] 
# uniques : 0.015477480266212661 10.526315789473683 0.1 0.1 
# shareds : 0.1547748026621266 75.177304964539 0.006976377974472255 0.9154449388600349 
# uq/shd : 0.01

In [8]:
print("Done!")

Done!
